# Load Model from Huggingface Hub

To use a model with the Transformer Extension one needs to download it from the Huggingface Hub and put it into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm). Please refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about model loading functionality it provides.

There are two ways of doing this.

1. Using the TE_MODEL_DOWNLOADER_UDF UDF.
2. Calling the CLI script `upload_model.py`.

Here we will use the first method.

<b>This notebook is not supposed to be run on its own. It contains model loading functions that are called by other notebooks.</b>

## Loading model with UDF

In [1]:
from exasol.nb_connector.secret_store import Secrets


def load_huggingface_model(conf: Secrets, model_name: str, task_type: str) -> None:
    """
    Loads specified model into BucketFS using a UDF provided with the Transformer Extension.

    conf           - Access to the secret store holding the configuration data.
    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    task_type      - Name of the task the model will be used for.
    """

    from exasol.nb_connector.connections import open_pyexasol_connection
    from exasol.nb_connector.language_container_activation import get_activation_sql
    from exasol.nb_connector.ai_lab_config import AILabConfig as CKey

    sql = f"""
    SELECT {conf.get(CKey.db_schema)}.TE_MODEL_DOWNLOADER_UDF(
        '{model_name}',
        '{task_type}',
        '{conf.get(CKey.te_models_bfs_dir)}',
        '{conf.get(CKey.te_bfs_connection)}',
        '{conf.get(CKey.te_hf_connection)}'
    )
    """

    with open_pyexasol_connection(conf, compression=True) as conn:
        conn.execute(query=get_activation_sql(conf))
        conn.execute(query=sql)